###
PIZZA EXAMPLE
###

In [1]:
!pip install gradio

import gradio as gr
from transformers import pipeline

# Initialize the zero-shot classifier (using facebook/bart-large-mnli)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Candidate labels now include an explicit "out_of_scope" option.
CANDIDATE_LABELS = ["greeting", "order_pizza", "menu_question", "out_of_scope"]

# Known pizza types for simple entity extraction simulation.
PIZZA_TYPES = ["pepperoni", "margherita", "vegetarian", "cheese"]

# Confidence threshold for deciding whether a prediction is reliable.
THRESHOLD = 0.7

def conversation_flow(user_input: str, state: dict):

    if state is None:
        state = {"stage": "initial", "order": None}

    text = user_input.strip().lower()

    # Check for immediate goodbye commands.
    if "bye" in text or "goodbye" in text:
        response = "Goodbye! Have a great day. (Resetting conversation.)"
        state = {"stage": "initial", "order": None}
        return response, state

    # Use zero-shot classification to evaluate the input.
    result = classifier(text, CANDIDATE_LABELS)
    top_label = result["labels"][0]
    top_score = result["scores"][0]

    # If confidence is too low or out_of_scope is predicted, return a fallback message.
    if top_score < THRESHOLD or top_label == "out_of_scope":
        response = "I'm not sure I understand. Could you please clarify your request?"
        return response, state

    # Stage: initial
    if state["stage"] == "initial":
        if top_label == "greeting":
            response = "Hello! How can I help you today? You can say something like 'I want to order a pizza.'"
            state["stage"] = "awaiting_intent"
        elif top_label == "order_pizza":
            response = "Great! What type of pizza would you like? (e.g., pepperoni, margherita)"
            state["stage"] = "awaiting_pizza_type"
        elif top_label == "menu_question":
            response = "Our menu includes pepperoni, margherita, vegetarian, and cheese pizzas. Would you like to place an order?"
            state["stage"] = "awaiting_intent"
        else:
            response = "I'm not sure I understand. Could you please clarify your request?"
            state["stage"] = "awaiting_intent"
        return response, state

    # Stage: awaiting_intent
    elif state["stage"] == "awaiting_intent":
        if top_label == "order_pizza":
            response = "Great! What type of pizza would you like? (e.g., pepperoni, margherita, vegetarian)"
            state["stage"] = "awaiting_pizza_type"
        elif top_label == "menu_question":
            response = "Our menu includes pepperoni, margherita, vegetarian, and cheese pizzas. Would you like to place an order?"
        elif top_label == "greeting":
            response = "Hi again! How can I assist you? If you'd like to order a pizza, just let me know."
        else:
            response = "Could you please specify your request? For example, say 'I want to order a pizza.'"
        return response, state

    # Stage: awaiting_pizza_type
    elif state["stage"] == "awaiting_pizza_type":
        found_type = None
        for pizza in PIZZA_TYPES:
            if pizza in text:
                found_type = pizza
                break
        if found_type:
            response = f"Got it! Your order for a {found_type} pizza has been placed. Thank you!"
            state["order"] = found_type
            state["stage"] = "order_confirmed"
        else:
            response = "I didn't catch the pizza type. Could you please specify (e.g., pepperoni, margherita, vegetarian, cheese)?"
        return response, state

    # Stage: order_confirmed
    elif state["stage"] == "order_confirmed":
        response = "Your order is confirmed. If you'd like to order another pizza or have more questions, just let me know!"
        # Reset state for new orders.
        state["stage"] = "awaiting_intent"
        state["order"] = None
        return response, state

    # Fallback: reset conversation.
    else:
        response = "I'm not sure how to proceed. Let's start over. How can I help you today?"
        state = {"stage": "initial", "order": None}
        return response, state

def chat_interface(user_message, history, state):

    if state is None:
        state = {"stage": "initial", "order": None}
    response, state = conversation_flow(user_message, state)
    history = history or []
    history.append((user_message, response))
    return history, state


with gr.Blocks() as demo_chat:
    gr.Markdown("# Pizza Ordering Chatbot")
    gr.Markdown(
        "This chatbot helps you order pizza"

    )
    chatbot = gr.Chatbot()
    state = gr.State({"stage": "initial", "order": None})
    txt = gr.Textbox(show_label=False, placeholder="Type your message here and press Enter")
    txt.submit(chat_interface, inputs=[txt, chatbot, state], outputs=[chatbot, state])


demo_chat.launch()



     |████████████████████████████████| 18.1 MB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 431 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 318 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 468 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 8.1 MB/s eta 0:00:011
     |████████████████████████████████| 100 kB 14.2 MB/s ta 0:00:01
     |████████████████████████████████| 45 kB 9.2 MB/s  eta 0:00:01
     |████████████████████████████████| 250 kB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 10.2 MB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 62 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 128 kB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 193 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 9.4 MB/s eta 0:00:01
     |████████████████████████████████| 71 

ModuleNotFoundError: No module named 'transformers'

In [2]:

!pip install transformers datasets evaluate

import json
import numpy as np
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate


with open("example.json", "r") as f:
    data = json.load(f)

# Extract texts and labels
texts = [item["text"] for item in data]
labels_str = [item["label"] for item in data]

# Create a label mapping (string-to-integer) based on unique labels
unique_labels = sorted(list(set(labels_str)))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

# Convert labels in the dataset to numeric form
numeric_labels = [label2id[label] for label in labels_str]

# Create a dictionary and then convert to a Hugging Face Dataset
dataset_dict = {"text": texts, "label": numeric_labels}
dataset = Dataset.from_dict(dataset_dict)

# Split the dataset into training (80%) and testing (20%) sets
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]



model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=64)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the dataset format to PyTorch tensors (only include needed columns)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


num_labels = len(unique_labels)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

"""
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluates at the end of each epoch.
    save_strategy="epoch",          # Saves the model at the end of each epoch.
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    seed=42,
    metric_for_best_model="accuracy"
)
"""
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",          # Save the model at the end of each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,             # Increased epochs from 3 to 5
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    seed=42,
    metric_for_best_model="accuracy"
)


# Define the evaluation metric: accuracy
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": acc}

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model on the test set
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

save_directory = "./pizza_model"
trainer.save_model(save_directory)
print(f"Model saved to {save_directory}")

     |████████████████████████████████| 10.2 MB 5.9 MB/s eta 0:00:01    |█████████████████████           | 6.7 MB 3.6 MB/s eta 0:00:01     |████████████████████████████▋   | 9.1 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 487 kB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 6.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 284 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 418 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 30.7 MB 8.0 MB/s eta 0:00:01
     |████████████████████████████████| 183 kB 16.9 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 16.3 MB/s eta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.3
    Uninstalling tqdm-4.62.3:
      Succes

ModuleNotFoundError: No module named 'torch'